In [1]:
% matplotlib inline
import sqlite3
import pandas as pd
import networkx as nx
import xlrd
import matplotlib.pyplot as plt
import math
import warnings
from tqdm import tqdm
import re
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### `c_office_chn` from UCI.

In [11]:
df_office_ming_drop_col=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSCmhbCk1B-9jjINMhy_VwikM6_Sn7bjdO7b_vaZJkVcYCCYlWVlhYVCFtAs0fPX-UEO62GWxaX1qAS/pub?gid=630627340&single=true&output=tsv',
                                    sep='\t')

In [12]:
df_office_ming_drop_col=df_office_ming_drop_col[['c_office_id（Dictionary Ser#)','c_office_chn']].rename(columns={'c_office_id（Dictionary Ser#)':'c_office_id'})

### `c_office_chn` from CBDB uncleaned.

In [2]:
conn = sqlite3.connect('../../SQL/20170424CBDBauUserSqlite.db')

In [3]:
df_office_ming=pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn)[pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn).c_dy==19]
df_office_ming_drop_col=df_office_ming[['c_office_id', 'c_office_chn']]
df_office_ming_drop_col.sample(5)

,c_office_id,c_office_chn
16575,71839,懷寧侯
16236,71500,正使
17469,72742,雲南司郎中管通州粮儲
17486,72759,管倉
15121,70385,國子學錄


### Coding `c_office_chn`.

In [13]:
df_adm=pd.read_csv('../data_dict/C_OT_ADM.tsv', sep='\t').set_index('c_ot_adm_id')
df_cls=pd.read_csv('../data_dict/C_OT_CLS.tsv', sep='\t').set_index('c_ot_cls_id')
df_tit=pd.read_csv('../data_dict/C_OT_TIT.tsv', sep='\t').set_index('c_ot_tit_id')

In [14]:
df_tit.sample(3)

,c_ot_tit_chinm,c_ot_tit_engnm,c_ot_tit_desc,c_ot_tit_start,c_ot_tit_end
c_ot_tit_id,,,,,
448,湖西道,NaN,NaN,NaN,NaN
336,撫寧伯,NaN,NaN,NaN,NaN
673,武勳王,NaN,NaN,NaN,NaN


In [15]:
# Replace titles (only one title in an office title string).
df_office_ming_drop_col['c_ot_coding']=''
for ming_ot_index in tqdm(df_office_ming_drop_col.index):
    ming_ot = df_office_ming_drop_col.loc[ming_ot_index, 'c_office_chn']
    for tit_index in df_tit.index:
        tit=df_tit.loc[tit_index, 'c_ot_tit_chinm']
        if ming_ot.endswith(tit):
            df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_tit_chinm']=tit
            df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_coding']=ming_ot.replace(tit, 'T'+str(tit_index))

100%|██████████| 4318/4318 [02:21<00:00, 30.43it/s]


In [16]:
df_office_ming_drop_col.sample(10)

,c_office_id,c_office_chn,c_ot_coding,c_ot_tit_chinm
414,1069,度支部員外郎,度支部T608,員外郎
2764,3054,倉大使,倉大T1338,使
2523,2931,鎮守總兵官,鎮守T234,總兵官
2951,71758,東川侯,東川T1348,侯
3908,638,尚寶兼守殿,,NaN
242,70665,驗封清吏司郎中,驗封清吏司T1297,郎中
3904,641,尚冠奉御,尚冠T1273,奉御
1195,71503,知貢舉官,T50,知貢舉官
4258,71650,左中允,T368,左中允
3503,95,大長公主,大長T971,公主


In [17]:
# Replace admin units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_drop_col.index):
    adm_list=[]
    for adm_index in df_adm.index:
        adm=df_adm.loc[adm_index, 'c_ot_adm_chinm']
        c_ot_coding = df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_coding']
        if adm in c_ot_coding:
            adm_list.append(adm)
            df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(adm, 'A'+str(adm_index))
    if adm_list!=[]:
        df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_adm_chinm']='#'.join(adm_list)

100%|██████████| 4318/4318 [03:04<00:00, 23.39it/s]


In [18]:
df_office_ming_drop_col.sample(10)

,c_office_id,c_office_chn,c_ot_coding,c_ot_tit_chinm,c_ot_adm_chinm
3648,226,太監,T1216,太監,NaN
3854,546,牲口房/僉書,A309/T1049,僉書,牲口房
2574,2954,分守參將,A825T931,參將,分守
2661,72108,/遊擊將軍,/A567T800,將軍,遊擊
3010,71831,鶴慶侯,鶴慶T1348,侯,NaN
4271,70913,右詹事,右T1165,詹事,NaN
1425,1928,參軍府左參軍,T1000A930左T1000,參軍,府
3761,248,管理太監,A808T1216,太監,管理
1397,2138,南鎮撫司鎮撫使,A151鎮撫T1338,使,南鎮撫司
3059,71993,衛國公,A925國T1342,公,衛


In [19]:
# Replace Classifications (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_drop_col.index):
    cls_list=[]
    for cls_index in df_cls.index:
        cls=df_cls.loc[cls_index, 'c_ot_cls_chinm']
        c_ot_coding = df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_coding']
        if cls in c_ot_coding:
            cls_list.append(cls)
            df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(cls, 'C'+str(cls_index))
    if cls_list!=[]:
        df_office_ming_drop_col.loc[ming_ot_index, 'c_ot_cls_chinm']='#'.join(cls_list)

100%|██████████| 4318/4318 [00:11<00:00, 383.24it/s]


In [20]:
df_office_ming_drop_col.to_excel('../dump/ming_office_title_coding_UCI.xlsx', encoding='utf8')